In [1]:
import numpy as np

class GridWorld:
    def __init__(self, size=5):
        self.size = size
        self.grid = np.zeros((size, size))  # Initialize grid
        self.agent_position = (0, 0)  # Agent starts at top-left
        self.goal_position = (size-1, size-1)  # Goal is at bottom-right
        self.obstacles = [(1, 1), (2, 2), (3, 3)]  # Obstacle positions
        self.actions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # Right, Down, Left, Up
        self.q_table = np.zeros((size, size, len(self.actions)))  # Q-table

    def get_valid_actions(self, position):
        valid_actions = []
        for action in self.actions:
            new_position = (position[0] + action[0], position[1] + action[1])
            if 0 <= new_position[0] < self.size and 0 <= new_position[1] < self.size \
                    and new_position not in self.obstacles:
                valid_actions.append(action)
        return valid_actions

    def take_action(self, position, action):
        new_position = (position[0] + action[0], position[1] + action[1])
        if new_position == self.goal_position:
            reward = 1  # Reached goal
        elif new_position in self.obstacles:
            reward = -1  # Hit obstacle
            new_position = position  # Stay in the same position
        else:
            reward = 0  # Normal movement
        return new_position, reward

    def train(self, episodes=100, alpha=0.1, gamma=0.9, epsilon=0.1):
        for _ in range(episodes):
            state = self.agent_position
            while state != self.goal_position:
                valid_actions = self.get_valid_actions(state)
                if np.random.uniform(0, 1) < epsilon:
                    action = valid_actions[np.random.randint(0, len(valid_actions))]
                else:
                    action = max(valid_actions, key=lambda a: self.q_table[state[0], state[1], self.actions.index(a)])
                next_state, reward = self.take_action(state, action)
                self.q_table[state[0], state[1], self.actions.index(action)] += \
                    alpha * (reward + gamma * np.max(self.q_table[next_state[0], next_state[1], :]) -
                             self.q_table[state[0], state[1], self.actions.index(action)])
                state = next_state
            self.agent_position = (0, 0)  # Reset agent position

    def test(self):
        state = self.agent_position
        path = [state]
        while state != self.goal_position:
            action = max(self.actions, key=lambda a: self.q_table[state[0], state[1], self.actions.index(a)])
            next_state, _ = self.take_action(state, action)
            path.append(next_state)
            state = next_state
        return path

# Initialize and train the gridworld
grid_world = GridWorld(size=5)
grid_world.train(episodes=1000)

# Test the trained model
path = grid_world.test()

print("Agent's path to reach the goal:")
for step, position in enumerate(path):
    print(f"Step {step+1}: {position}")



Agent's path to reach the goal:
Step 1: (0, 0)
Step 2: (0, 1)
Step 3: (0, 2)
Step 4: (0, 3)
Step 5: (0, 4)
Step 6: (1, 4)
Step 7: (2, 4)
Step 8: (3, 4)
Step 9: (4, 4)
